# Causal inference tutorial: A simple example
Estimate the causal effect of a recommender system based on logs. 

In [2]:
## Script to estimate causal effect of recommendations on a website.

# Library for easy manipulation of data frames.
import numpy as np
import pandas as pd

# A parameter for the number of recommendations shown to the user
MAX_SHOWN_RECS = 3

## Different causal inference estimation methods

In [3]:
def naive_observational_estimate(user_visits):
  # Naive observational estimate
  # Simply the fraction of visits that resulted in a recommendation click-through.
  est = np.sum(user_visits["is_rec_visit"])/len(user_visits["is_rec_visit"])
  print("Mean estimate: {0}".format(est))
  return(est)

def stratified_by_activity_estimate(user_visits):
  # Stratified observational estimate by activity level of each user.
  grouped = user_visits.groupby('activity_level')
  grouped = grouped.agg({'is_rec_visit': lambda x: np.sum(x)/x.size })
  est = grouped.rename(columns= {'is_rec_visit':'stratified_estimate'})
  print("Mean estimate: {0}".format(np.mean(est['stratified_estimate'])))
  return(est)

def stratified_by_category_estimate(user_visits):
  # Stratified observational estimate by app category 
  grouped = user_visits.groupby('category')
  grouped = grouped.agg({'is_rec_visit': lambda x: np.sum(x)/x.size })
  est = grouped.rename(columns= {'is_rec_visit':'stratified_estimate'})
  print("Mean estimate: {0}".format(np.mean(est['stratified_estimate'])))
  return(est)

def fully_conditioned_estimate(user_visits):
  # Stratified observational estimate by both user activity level and app category.
  grouped = user_visits.groupby(['activity_level', 'category'])
  grouped = grouped.agg({'is_rec_visit': lambda x: np.sum(x)/x.size })
  est = grouped.rename(columns= {'is_rec_visit':'stratified_estimate'})
  print("Mean estimate: {0}".format(np.mean(est['stratified_estimate'])))
  return(est)

def ranking_discontinuity_estimate(user_visits):
  # Regression discontinuity estimate for the causal effect of recommendation system.
  ctr_by_rank = user_visits.groupby('rec_rank')
  ctr_by_rank = ctr_by_rank.agg({'product_id': lambda x: x.size})
  ctr_by_rank = ctr_by_rank.rename(columns={'product_id':'num_clicks_by_rank'})
  ctr_by_rank = ctr_by_rank.reset_index()
  sum_num_clicks = np.sum(ctr_by_rank['num_clicks_by_rank'])
  ctr_by_rank['ctr_estimate_by_rank'] = ctr_by_rank['num_clicks_by_rank']/sum_num_clicks

  
  # Comparing the last shown recommendation and the first not-shown recommendation.
  # Assuming that there are no position order effects in recommendation click-throughs.
  print(ctr_by_rank)
  est1 = ctr_by_rank.loc[ctr_by_rank.rec_rank==MAX_SHOWN_RECS, 'ctr_estimate_by_rank']
  est2 = ctr_by_rank.loc[ctr_by_rank.rec_rank==(MAX_SHOWN_RECS+1), 'ctr_estimate_by_rank']
  est = est1.iloc[0] -est2.iloc[0]
  upper_bound_est = est*MAX_SHOWN_RECS
  print("Mean estimate: {0}".format(upper_bound_est))
  return(upper_bound_est)

##  Reading app visit logs for two different algorithms.

In [4]:
user_app_visits_A = pd.read_csv("../datasets/user_app_visits_A.csv")
user_app_visits_B = pd.read_csv("../datasets/user_app_visits_B.csv")

### GOAL 1: COMPARE ALGORITHM A VERSUS B
**Naive estimate**

In [5]:
naive_observational_estimate(user_app_visits_A)
naive_observational_estimate(user_app_visits_B) 

Mean estimate: 0.201311
Mean estimate: 0.225052


0.225052

 **Stratified estimate** (by user activity level)

In [6]:

print(stratified_by_activity_estimate(user_app_visits_A))
print(stratified_by_activity_estimate(user_app_visits_B))

Mean estimate: 0.20035698127413026
                stratified_estimate
activity_level                     
1                          0.125651
2                          0.175509
3                          0.225762
4                          0.274506
Mean estimate: 0.20012010666246788
                stratified_estimate
activity_level                     
1                          0.124835
2                          0.175152
3                          0.225146
4                          0.275348


 **Stratified estimate** (by app category)

In [7]:
print(stratified_by_category_estimate(user_app_visits_A))
print(stratified_by_category_estimate(user_app_visits_B))

Mean estimate: 0.20126888085162636
          stratified_estimate
category                     
1                    0.174848
2                    0.224932
3                    0.276092
4                    0.127721
5                    0.178281
6                    0.227305
7                    0.275923
8                    0.123859
9                    0.176174
10                   0.227554
Mean estimate: 0.22496994851719032
          stratified_estimate
category                     
1                    0.200286
2                    0.250328
3                    0.297486
4                    0.150637
5                    0.200782
6                    0.250557
7                    0.301367
8                    0.149972
9                    0.198176
10                   0.250108


In [8]:
print(fully_conditioned_estimate(user_app_visits_A))
print(fully_conditioned_estimate(user_app_visits_B))  

Mean estimate: 0.2003106574488021
                         stratified_estimate
activity_level category                     
1              1                    0.100107
               2                    0.151534
               3                    0.197995
               4                    0.050482
               5                    0.101296
               6                    0.151942
               7                    0.196753
               8                    0.050324
               9                    0.101892
               10                   0.155373
2              1                    0.148901
               2                    0.195862
               3                    0.251806
               4                    0.102751
               5                    0.150518
               6                    0.200401
               7                    0.252204
               8                    0.099486
               9                    0.149651
               10    

In [10]:
### Ranking Discontinuity Estimate makes sense. 
### The last shown and first not shown item

# GOAL 2: FIND THE CAUSAL EFFECT OF SHOWING RECOMMENDATIONS
# Regression discontinuity estimate for Algorithm A
print(naive_observational_estimate(user_app_visits_A))
ranking_discontinuity_estimate(user_app_visits_A)


Mean estimate: 0.201311
0.201311
   rec_rank  num_clicks_by_rank  ctr_estimate_by_rank
0        -1              718766              0.718766
1         1               66911              0.066911
2         2               67216              0.067216
3         3               67184              0.067184
4         4               26741              0.026741
5         5               26666              0.026666
6         6               26516              0.026516
Mean estimate: 0.12132899999999998


0.12132899999999998